In [1]:
import bayesfast as bf
import numpy as np

In [2]:
np.random.seed(0)
D = 10
N = 10000
a = 1.
b = 0.5

def logp(x):
    n = x.shape[-1]
    _a = -0.5 * x[..., 0]**2 / a**2
    _b = -0.5 * np.sum(x[..., 1:]**2, axis=-1) * np.exp(-2 * b * x[..., 0])
    _c = (-0.5 * np.log(2 * np.pi * a**2) - 
          0.5 * (n - 1) * np.log(2 * np.pi) - (n - 1) * b * x[..., 0])
    return _a + _b + _c

xp = np.random.randn(N, D)
xp[:, 0] *= a
xp[:, 1:] *= np.exp(b * xp[:, :1])

In [3]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=10, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:46331 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 10 Cores: 10 Memory: 540.15 GB


In [4]:
from bayesfast.transforms import INT

r = INT(xp, client=client, random_state=1, ica_kwargs={'max_iter': 50},
        random_method='pseudo')

In [5]:
r.iterate(10)

/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/global/homes/h/hejia/.conda/envs/hejia@cori-2/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/glo

In [6]:
xq, _, _ = r.sample(N)

In [7]:
logp_xp = logp(xp)
logp_xq = logp(xq)
logq_xp = r.logq(xp)
logq_xq = r.logq(xq)

In [8]:
from bayesfast.evidence.bridge import BS
res = BS(logp_xp=logp_xp, logp_xq=logp_xq, logq_xp=logq_xp, logq_xq=logq_xq)

In [9]:
res[:2] # logZ, logZ_err

(-0.09788450134889844, 0.008638439113154392)